In [31]:

pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [32]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
import os

In [33]:
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-18.0.2.1"

In [34]:
conf = SparkConf() \
    .setAppName("project") \
    .setMaster("local") \
    .set("spark.driver.extraClassPath","C:/pyspark/*")

In [35]:
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)

In [36]:
spark

In [37]:
database = "uber"
table = "fact_table"
password = ""
user = "postgres"
schema  = "analytics"

In [38]:
jdbc_url = 'jdbc:postgresql://localhost:5432/uber?user=postgres&password=<>'



In [39]:
df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", f"{schema}.{table}") \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .load()



In [40]:
df.show()

+-----+--------+-----------+------------------+----------------+------------+------------------+--------------+--------------+---------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|index|VendorID|datetime_id|passenger_count_id|trip_distance_id|rate_code_id|store_and_fwd_flag|pu_location_id|do_location_id|payment_type_id|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+-----+--------+-----------+------------------+----------------+------------+------------------+--------------+--------------+---------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|    0|       1|         11|                11|              11|          11|                 N|            11|            11|             11|       33.8|  3.5|    0.5|      7.75|         0.0|                  1.0|   

In [41]:
df.limit(20).toPandas()

,index,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pu_location_id,do_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,0,1,11,11,11,11,N,11,11,11,33.8,3.5,0.5,7.75,0.00,1.0,46.55,2.5,0.0
1,1,1,17,17,17,17,N,17,17,17,20.5,3.5,0.5,4.00,0.00,1.0,29.50,2.5,0.0
2,2,1,18,18,18,18,N,18,18,18,8.6,3.5,0.5,0.00,0.00,1.0,13.60,2.5,0.0
3,3,1,19,19,19,19,N,19,19,19,15.6,3.5,0.5,0.00,0.00,1.0,20.60,2.5,0.0
4,4,1,20,20,20,20,N,20,20,20,9.3,3.5,0.5,2.85,0.00,1.0,17.15,2.5,0.0
5,5,1,22,22,22,22,N,22,22,22,8.6,3.5,0.5,0.00,0.00,1.0,13.60,2.5,0.0
6,6,1,23,23,23,23,N,23,23,23,11.4,3.5,0.5,0.00,0.00,1.0,16.40,2.5,0.0
7,7,1,27,27,27,27,N,27,27,27,13.5,3.5,0.5,3.70,0.00,1.0,22.20,2.5,0.0
8,8,1,28,28,28,28,N,28,28,28,5.1,3.5,0.5,1.82,0.00,1.0,11.92,2.5,0.0
9,9,1,29,29,29,29,N,29,29,29,7.9,3.5,0.5,2.55,0.00,1.0,15.45,2.5,0.0


# rename column


In [42]:
df = df.withColumnRenamed("passenger_count_id", "datetime_id")
df.show

<bound method DataFrame.show of DataFrame[index: bigint, VendorID: bigint, datetime_id: bigint, datetime_id: bigint, trip_distance_id: bigint, rate_code_id: bigint, store_and_fwd_flag: string, pu_location_id: bigint, do_location_id: bigint, payment_type_id: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, airport_fee: double]>

In [43]:
df.count()

100

In [44]:
df = df.select("VendorID", "pu_location_id", "congestion_surcharge", "rate_code_id", "payment_type_id", "")
df.show()

+--------+--------------+--------------------+------------+---------------+
|VendorID|pu_location_id|congestion_surcharge|rate_code_id|payment_type_id|
+--------+--------------+--------------------+------------+---------------+
|       1|            11|                 2.5|          11|             11|
|       1|            17|                 2.5|          17|             17|
|       1|            18|                 2.5|          18|             18|
|       1|            19|                 2.5|          19|             19|
|       1|            20|                 2.5|          20|             20|
|       1|            22|                 2.5|          22|             22|
|       1|            23|                 2.5|          23|             23|
|       1|            27|                 2.5|          27|             27|
|       1|            28|                 2.5|          28|             28|
|       1|            29|                 2.5|          29|             29|
|       1|  

In [45]:
df.sort("fare_amount").show()

+--------+--------------+--------------------+------------+---------------+
|VendorID|pu_location_id|congestion_surcharge|rate_code_id|payment_type_id|
+--------+--------------+--------------------+------------+---------------+
|       2|            61|                 2.5|          61|             61|
|       2|            95|                 0.0|          95|             95|
|       1|            69|                 2.5|          69|             69|
|       1|            71|                 2.5|          71|             71|
|       1|            28|                 2.5|          28|             28|
|       1|            49|                 2.5|          49|             49|
|       2|            35|                 2.5|          35|             35|
|       2|            77|                 2.5|          77|             77|
|       2|            43|                 2.5|          43|             43|
|       2|            45|                 2.5|          45|             45|
|       1|  

# sort as descending

In [46]:
from pyspark.sql import functions as F
df.sort(F.desc("fare_amount")).show()

+--------+--------------+--------------------+------------+---------------+
|VendorID|pu_location_id|congestion_surcharge|rate_code_id|payment_type_id|
+--------+--------------+--------------------+------------+---------------+
|       1|            21|                 0.0|          21|             21|
|       2|            91|                 0.0|          91|             91|
|       2|            59|                 0.0|          59|             59|
|       2|            90|                 0.0|          90|             90|
|       2|            89|                 2.5|          89|             89|
|       1|            64|                 2.5|          64|             64|
|       2|            48|                 0.0|          48|             48|
|       2|             7|                 2.5|           7|              7|
|       2|            13|                 0.0|          13|             13|
|       2|            66|                 2.5|          66|             66|
|       1|  

In [47]:
df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- pu_location_id: long (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- rate_code_id: long (nullable = true)
 |-- payment_type_id: long (nullable = true)



# DataFrame filtering

In [48]:
df.select("pu_location_id", "congestion_surcharge").filter('congestion_surcharge = "0.0"').show()


+--------------+--------------------+
|pu_location_id|congestion_surcharge|
+--------------+--------------------+
|            14|                 0.0|
|            95|                 0.0|
|            13|                 0.0|
|            96|                 0.0|
|             3|                 0.0|
|            48|                 0.0|
|            59|                 0.0|
|            90|                 0.0|
|            91|                 0.0|
|            97|                 0.0|
|            21|                 0.0|
+--------------+--------------------+



In [49]:
df.select(df.pu_location_id, df.congestion_surcharge).filter('congestion_surcharge = "0.0"').show()

+--------------+--------------------+
|pu_location_id|congestion_surcharge|
+--------------+--------------------+
|            14|                 0.0|
|            95|                 0.0|
|            13|                 0.0|
|            96|                 0.0|
|             3|                 0.0|
|            48|                 0.0|
|            59|                 0.0|
|            90|                 0.0|
|            91|                 0.0|
|            97|                 0.0|
|            21|                 0.0|
+--------------+--------------------+



In [50]:
df.select("pu_location_id", "congestion_surcharge").filter("congestion_surcharge like '%0.0%'").show()

+--------------+--------------------+
|pu_location_id|congestion_surcharge|
+--------------+--------------------+
|            14|                 0.0|
|            95|                 0.0|
|            13|                 0.0|
|            96|                 0.0|
|             3|                 0.0|
|            48|                 0.0|
|            59|                 0.0|
|            90|                 0.0|
|            91|                 0.0|
|            97|                 0.0|
|            21|                 0.0|
+--------------+--------------------+



In [51]:
df.filter((df.pu_location_id.like ('%14%')) & (df.congestion_surcharge=='0.0')).show()

+--------+--------------+--------------------+------------+---------------+
|VendorID|pu_location_id|congestion_surcharge|rate_code_id|payment_type_id|
+--------+--------------+--------------------+------------+---------------+
|       2|            14|                 0.0|          14|             14|
+--------+--------------+--------------------+------------+---------------+



# To Crate a spark SQL create a view

In [52]:
df.createOrReplaceTempView("fact_table")
spark.sql("select count(1) from fact_table").show()

+--------+
|count(1)|
+--------+
|     100|
+--------+



In [53]:
spark.sql("select pu_location_id, congestion_surcharge from fact_table where congestion_surcharge = '0.0'").show()

+--------------+--------------------+
|pu_location_id|congestion_surcharge|
+--------------+--------------------+
|            14|                 0.0|
|            95|                 0.0|
|            13|                 0.0|
|            96|                 0.0|
|             3|                 0.0|
|            48|                 0.0|
|            59|                 0.0|
|            90|                 0.0|
|            91|                 0.0|
|            97|                 0.0|
|            21|                 0.0|
+--------------+--------------------+



# Importing a new dataset to join with older dataset

In [54]:
dim_payment_type = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", f"analytics.dim_payment_type") \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "org.postgresql.Driver") \
    .load()


In [55]:
dim_payment_type.show()

+-----+---------------+------------+-----------------+
|index|payment_type_id|payment_type|payment_type_name|
+-----+---------------+------------+-----------------+
|    0|              1|           1|      Credit card|
|    1|              2|           1|      Credit card|
|    2|              3|           1|      Credit card|
|    3|              4|           1|      Credit card|
|    4|              5|           1|      Credit card|
|    5|              6|           1|      Credit card|
|    6|              7|           1|      Credit card|
|    7|              8|           1|      Credit card|
|    8|             10|           1|      Credit card|
|    9|             11|           1|      Credit card|
|   10|             12|           1|      Credit card|
|   11|             13|           1|      Credit card|
|   12|             14|           1|      Credit card|
|   13|             15|           1|      Credit card|
|   14|             16|           1|      Credit card|
|   15|   

In [56]:
#caching for better performance
dim_payment_type.createOrReplaceTempView("dim_payment_type")
dim_payment_type.cache()

DataFrame[index: bigint, payment_type_id: bigint, payment_type: bigint, payment_type_name: string]

In [57]:
spark.sql("""SELECT * FROM dim_payment_type""").show()

+-----+---------------+------------+-----------------+
|index|payment_type_id|payment_type|payment_type_name|
+-----+---------------+------------+-----------------+
|    0|              1|           1|      Credit card|
|    1|              2|           1|      Credit card|
|    2|              3|           1|      Credit card|
|    3|              4|           1|      Credit card|
|    4|              5|           1|      Credit card|
|    5|              6|           1|      Credit card|
|    6|              7|           1|      Credit card|
|    7|              8|           1|      Credit card|
|    8|             10|           1|      Credit card|
|    9|             11|           1|      Credit card|
|   10|             12|           1|      Credit card|
|   11|             13|           1|      Credit card|
|   12|             14|           1|      Credit card|
|   13|             15|           1|      Credit card|
|   14|             16|           1|      Credit card|
|   15|   

In [58]:
spark.sql("""SELECT * FROM fact_table""").show()

+--------+--------------+--------------------+------------+---------------+
|VendorID|pu_location_id|congestion_surcharge|rate_code_id|payment_type_id|
+--------+--------------+--------------------+------------+---------------+
|       1|            11|                 2.5|          11|             11|
|       1|            17|                 2.5|          17|             17|
|       1|            18|                 2.5|          18|             18|
|       1|            19|                 2.5|          19|             19|
|       1|            20|                 2.5|          20|             20|
|       1|            22|                 2.5|          22|             22|
|       1|            23|                 2.5|          23|             23|
|       1|            27|                 2.5|          27|             27|
|       1|            28|                 2.5|          28|             28|
|       1|            29|                 2.5|          29|             29|
|       1|  

In [63]:
#join with sparkSQL
spark.sql("""
SELECT 
	f.pu_location_id,
    p.payment_type
FROM fact_table f
    INNER JOIN dim_payment_type p ON f.payment_type_id = p.payment_type_id
WHERE p.payment_type_name LIKE '%Credit card%'
"""
         ).show()


+--------------+------------+
|pu_location_id|payment_type|
+--------------+------------+
|            11|           1|
|            17|           1|
|            20|           1|
|            27|           1|
|            28|           1|
|            29|           1|
|            31|           1|
|            51|           1|
|            54|           1|
|            64|           1|
|            67|           1|
|            68|           1|
|            69|           1|
|            70|           1|
|            71|           1|
|            72|           1|
|            92|           1|
|            93|           1|
|            94|           1|
|             1|           1|
+--------------+------------+
only showing top 20 rows



In [79]:
#Join with spark dataframe
joining = dim_payment_type.join(df, ['payment_type_id'], how='inner')
joining.limit(100).toPandas()

,payment_type_id,index,payment_type,payment_type_name,how_I_was_paid,VendorID,pu_location_id,congestion_surcharge,rate_code_id
0,11,9,1,Credit card,ATM,1,11,2.5,11
1,17,15,1,Credit card,ATM,1,17,2.5,17
2,18,80,2,Cash,money,1,18,2.5,18
3,19,81,2,Cash,money,1,19,2.5,19
4,20,16,1,Credit card,ATM,1,20,2.5,20
...,...,...,...,...,...,...,...,...,...
95,59,90,2,Cash,money,2,59,0.0,59
96,90,70,1,Credit card,ATM,2,90,0.0,90
97,91,71,1,Credit card,ATM,2,91,0.0,91
98,97,75,1,Credit card,ATM,2,97,0.0,97


In [68]:
joining.groupBy(['payment_type_id']).agg(
    F.sum('rate_code_id').alias('Total_rate_code'),\
    F.max('rate_code_id').alias('Max_rate_code')\
    ).show()

+---------------+---------------+-------------+
|payment_type_id|Total_rate_code|Max_rate_code|
+---------------+---------------+-------------+
|             29|             29|           29|
|             26|             26|           26|
|             65|             65|           65|
|             19|             19|           19|
|             54|             54|           54|
|              0|              0|            0|
|             22|             22|           22|
|              7|              7|            7|
|             77|             77|           77|
|             34|             34|           34|
|             50|             50|           50|
|             94|             94|           94|
|             57|             57|           57|
|             32|             32|           32|
|             43|             43|           43|
|             84|             84|           84|
|             31|             31|           31|
|             39|             39|       

In [70]:
#create a new column with calculation
calc_net_charge = joining.withColumn('net_charge', F.col('congestion_surcharge') - F.col('rate_code_id'))
calc_net_charge.limit(10).toPandas()

,payment_type_id,index,payment_type,payment_type_name,VendorID,pu_location_id,congestion_surcharge,rate_code_id,net_charge
0,11,9,1,Credit card,1,11,2.5,11,-8.5
1,17,15,1,Credit card,1,17,2.5,17,-14.5
2,18,80,2,Cash,1,18,2.5,18,-15.5
3,19,81,2,Cash,1,19,2.5,19,-16.5
4,20,16,1,Credit card,1,20,2.5,20,-17.5
5,22,98,4,Dispute,1,22,2.5,22,-19.5
6,23,99,4,Dispute,1,23,2.5,23,-20.5
7,27,21,1,Credit card,1,27,2.5,27,-24.5
8,28,22,1,Credit card,1,28,2.5,28,-25.5
9,29,23,1,Credit card,1,29,2.5,29,-26.5


In [77]:
#creating a new column based on condition
dim_payment_type = dim_payment_type.withColumn(
    'how_I_was_paid',
    F.when((F.col('payment_type_name') == 'Credit card'), 'ATM')
    .when((F.col('payment_type_name') == 'Cash'), 'money')
    .otherwise('dispute')
)
dim_payment_type.limit(100).toPandas()


,index,payment_type_id,payment_type,payment_type_name,how_I_was_paid
0,0,1,1,Credit card,ATM
1,1,2,1,Credit card,ATM
2,2,3,1,Credit card,ATM
3,3,4,1,Credit card,ATM
4,4,5,1,Credit card,ATM
...,...,...,...,...,...
95,95,88,2,Cash,money
96,96,95,2,Cash,money
97,97,96,2,Cash,money
98,98,22,4,Dispute,dispute


In [78]:
#checking distinct / unique values
dim_payment_type.select('how_I_was_paid').distinct().collect()

[Row(how_I_was_paid='ATM'),
 Row(how_I_was_paid='money'),
 Row(how_I_was_paid='dispute')]

In [82]:
#replacing null values with
from pyspark.sql.functions import when, lit

df = df.withColumn('congestion_surcharge', when(df.congestion_surcharge.isNull(),
                                                                           lit(0)).otherwise(df.congestion_surcharge))

df.limit(100).toPandas()


,VendorID,pu_location_id,congestion_surcharge,rate_code_id,payment_type_id
0,1,11,2.5,11,11
1,1,17,2.5,17,17
2,1,18,2.5,18,18
3,1,19,2.5,19,19
4,1,20,2.5,20,20
...,...,...,...,...,...
95,2,59,0.0,59,59
96,2,90,0.0,90,90
97,2,91,0.0,91,91
98,2,97,0.0,97,97


In [83]:
#drop a column
df = df.drop('store_and_fwd_flag')
df.limit(100).toPandas()


,VendorID,pu_location_id,congestion_surcharge,rate_code_id,payment_type_id
0,1,11,2.5,11,11
1,1,17,2.5,17,17
2,1,18,2.5,18,18
3,1,19,2.5,19,19
4,1,20,2.5,20,20
...,...,...,...,...,...
95,2,59,0.0,59,59
96,2,90,0.0,90,90
97,2,91,0.0,91,91
98,2,97,0.0,97,97
